# Paillier Encription:

## Key Generation

In [25]:
import random
from sympy import mod_inverse, lcm

In [26]:
def gcd(a, b):
    while b != 0:
        a, b = b, a % b
    return a

In [27]:
def L(u, n):
    return (u - 1) // n

In [28]:
def is_prime(n):
    if n <= 1:
        return False
    for i in range(2, int(n**0.5) + 1):
        if n % i == 0:
            return False
    return True

In [29]:
def random_prime(bits):
    while True:
        p = random.getrandbits(bits)        
        if is_prime(p):
            return p

In [30]:
def is_valid_g(g, n, n2, lam):
    # Check gcd(L(g^lam mod n^2), n) == 1
    if g < 1 or g >= n2:
        return False
    l = L(pow(g, lam, n2), n)
    return gcd(l, n) == 1


In [31]:
def generate_keypair(bits):
    # Generate two large primes p and q
    p = random_prime(bits // 2)
    print(f"p: {p}")
    q = random_prime(bits // 2)
    print(f"q: {q}")
    
    n = p * q
    print(f"n: {n}")
    n2 = n * n
    lam = int(lcm(p - 1, q - 1))
    print(f"lamda: {lam}")    

    # Choose g in Z*_{n^2}
    g = random.randint(1, n2 - 1)    
    while not is_valid_g(g, n, n2, lam):
        g = random.randint(1, n2 - 1)
        
    print(f"g: {g}")

    # Compute mu
    mu = mod_inverse(L(pow(g, lam, n2), n), n)
    print(f"mu: {mu}")

    # Public key (n, g)
    public_key = (n, g)    
    
    # Private key (lam, mu)
    private_key = (lam, mu)

    return public_key, private_key


## Encryption

In [32]:
def encrypt(public_key, plaintext):
    n, g = public_key
    n2 = n * n
    r = random.randint(1, n - 1)
    while gcd(r, n) != 1:
        r = random.randint(1, n - 1)
    c = (pow(g, plaintext, n2) * pow(r, n, n2)) % n2
    return c


## Decryption

In [33]:
def decrypt(private_key, public_key, ciphertext):
    lam, mu = private_key
    n, g = public_key
    n2 = n * n
    u = pow(ciphertext, lam, n2)
    l = L(u, n)
    plaintext = (l * mu) % n
    return plaintext

## operation

In [83]:
def homomorphic_addition(ciphertext1, ciphertext2, n2):
    return (ciphertext1 * ciphertext2) % n2

In [89]:
def homomorphic_scalar_multiplication(ciphertext, scalar, n2):
    return pow(ciphertext, scalar, n2)

In [90]:
def homomorphic_negation(ciphertext, n2):     
    return mod_inverse(ciphertext, n2)

def homomorphic_subtraction(ciphertext1, ciphertext2, n):    
    neg_ciphertext2 = homomorphic_negation(ciphertext2, n)
    return homomorphic_addition(ciphertext1, neg_ciphertext2, n**2)

# Test

In [93]:
# Example Usage
bits = 16
public_key, private_key = generate_keypair(bits)
n = public_key[0] ** 2
n2 = n ** 2
print("Public Key:", public_key)
print("Private Key:", private_key)


plaintext1 = 123
plaintext2 = 456

# Encrypt both plaintexts
ciphertext1 = encrypt(public_key, plaintext1)
ciphertext2 = encrypt(public_key, plaintext2)

# Perform homomorphic addition on the ciphertexts
ciphertext_sum = homomorphic_addition(ciphertext1, ciphertext2, n2)

# Decrypt the result
decrypted_sum = decrypt(private_key, public_key, ciphertext_sum)

print("Plaintext 1:", plaintext1)
print("Plaintext 2:", plaintext2)
print("Encrypted Plaintext 1:", ciphertext1)
print("Encrypted Plaintext 2:", ciphertext2)
print("Encrypted Sum:", ciphertext_sum)
print("Decrypted Sum:", decrypted_sum)

decrypt(private_key, public_key, homomorphic_subtraction(ciphertext2, ciphertext1, n))

p: 193
q: 83
n: 16019
lamda: 7872
g: 91860186
mu: 13344
Public Key: (16019, 91860186)
Private Key: (7872, 13344)
Plaintext 1: 123
Plaintext 2: 456
Encrypted Plaintext 1: 183689278
Encrypted Plaintext 2: 159681387
Encrypted Sum: 29331758688068586
Decrypted Sum: 579


333